In [4]:
import numpy as np


K = GF(9, 'alpha')
alpha = K.gen() 
d = K.order() // 2 + 1  


def check_inequality(n, r):
    return (r == n) 


def make_weight1(n, K):
    slots = list(range(n))
    P = matrix(K, n, n, 0)
    for i in range(n):
        index = choice(slots)
        slots.remove(index)
        P[i, index] = 1
    return P


def Schur_product(M, k1):
    Schur = []
    for i in range(k1):
        row_i = np.array(list(M[i]))
        for j in range(i, k1):
            row_j = np.array(list(M[j]))
            Schur.append(list(np.multiply(row_i, row_j)))
    return Schur


def generate_random_non_singular_matrix(K, k1):
    S = random_matrix(K, k1, k1)
    while S.is_singular():
        S = random_matrix(K, k1, k1)
    return S


def SchurCalc(M, K):
    k1 = M.rank()
    n = M.ncols()
    S = generate_random_non_singular_matrix(K, k1)
    P1 = make_weight1(n, K)
    G_cap_hidden1 = S * M * P1
    G_cap_Schur1 = Matrix(K, Schur_product(G_cap_hidden1, k1))
    G1k = G_cap_Schur1.rank()
    return check_inequality(n, G1k)


def codegen(K, d, a, b):
    res = [None] * 5
    disc = K(-16) * (K(4) * a**3 + K(27) * b**2)
    if disc == 0:
        raise ValueError("Сингулярная кривая: дискриминант равен 0")
    # кривая
    R.<x, y> = PolynomialRing(K, 2)
    f = y**2 - (x**3 + a * x + b)
    C = Curve(f)
    # функциональное поле кривой и места порядка 1
    F = C.function_field()
    places = F.places(1)
    if len(places) <= 1:
        raise ValueError("Недостаточно мест для построения AG-кода")
    # дивизор 
    Q = places[0]
    del places[0]
    G = d * Q
    # AG-код
    code = codes.EvaluationAGCode(places, G)
    res[0] = SchurCalc(code.generator_matrix(), K)
    # Проколотый код
    try:
        code1 = codes.PuncturedCode(code, 1)
        res[1] = SchurCalc(code1.generator_matrix(), K)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить проколотый")
    # Укороченный код
    try:
        code2 = code1.shortened([1])
        res[2] = SchurCalc(code2.generator_matrix(), K)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить укороченный")
    # Подполевой код
    code3 = codes.SubfieldSubcode(code, K)
    res[3] = SchurCalc(code3.generator_matrix(), K)
    # Трэйс-код
    code41 = code.dual_code()
    code42 = codes.SubfieldSubcode(code41, K)
    code43 = code42.dual_code()
    res[4] = SchurCalc(code43.generator_matrix(), K)
    return res


# -------------------- ОСНОВНОЙ БЛОК --------------------
number_tests = 0    # количество успешных тестов
probs = [0] * 5     # средние значения (доли успеха) для пяти типов кодов
total_tests = 10   # количество попыток

while number_tests < total_tests:
    # генерируем коэффициенты a и b 
    a = K.random_element()
    b = K.random_element()
    try:
        res = codegen(K, d, a, b)
        if None not in res:
            for i in range(5):
                probs[i] = (probs[i] * number_tests + (1 if res[i] else 0)) / (number_tests + 1)
            number_tests += 1
    except Exception as e:
        print("error is", e)
        continue

code_names = ['AG-code', 'Punctured-code', 'Shortened-code', 'Subfield-code', 'Trace-code']
for i in range(5):
    print(f"{code_names[i]}: {probs[i]}")

AG-code: 9/10
Punctured-code: 9/10
Shortened-code: 9/10
Subfield-code: 9/10
Trace-code: 9/10
